In [ ]:
%pip install comet_ml
from comet_ml import Experiment
from comet_ml.integration.pytorch import log_model
experiment = Experiment(
  api_key = "g208he4vUVLoAY4sFmdPGDBDO",
  project_name = "Monitorización Final",
  workspace="kpstufforigins",
  log_code=True,
  log_graph=True,
)
experiment.add_tag("CycleGAN 1")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 549.6/549.6 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.7/214.7 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.1/510.1 kB 21.0 MB/s eta 0:00:00
  Attempting uninstall: websocket-client
    Found existing installation: websocket-client 1.6.1
    Uninstalling websocket-client-1.6.1:
      Successfully uninstalled websocket-client-1.6.1


COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Couldn't find a Git repository in '/content' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.
COMET INFO: Experiment is live on comet.com https://www.comet.com/kpstufforigins/monitorizaci-n-final/158635b96892414daaa1c430d14166ad



In [ ]:
from google.colab import drive
import zipfile
drive.mount('/content/drive')
with zipfile.ZipFile('/content/drive/MyDrive/TFG/UTK_Corto.zip', 'r') as zip_ref:
    zip_ref.extractall('.')

Mounted at /content/drive


In [ ]:
import torch
import albumentations as A
from albumentations.pytorch import ToTensorV2

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
TRAIN_DIR = "."
VAL_DIR = "."
BATCH_SIZE = 4
LEARNING_RATE = 1e-4
LAMBDA_IDENTITY = 0.02
LAMBDA_CYCLE = 10
NUM_WORKERS = 4
NUM_EPOCHS = 1500
LOAD_MODEL = False
SAVE_MODEL = True
CHECKPOINT_GEN_H = "genh.pth.tar"
CHECKPOINT_GEN_Z = "genz.pth.tar"
CHECKPOINT_CRITIC_H = "critich.pth.tar"
CHECKPOINT_CRITIC_Z = "criticz.pth.tar"

transforms = A.Compose(
    [
        A.Resize(width=256, height=256),
        A.HorizontalFlip(p=0.5),
        A.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5], max_pixel_value=255),
        ToTensorV2(),
    ],
    additional_targets={"image0": "image"},
)
from prettytable import PrettyTable

def count_parameters(model):
    table = PrettyTable(["Modules", "Parameters"])
    total_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad:
            continue
        params = parameter.numel()
        table.add_row([name, params])
        total_params += params
    print(table)
    print(f"Total Trainable Params: {total_params}")
    return total_params

In [ ]:
from PIL import Image
import os
from torch.utils.data import Dataset
import numpy as np

class HorseZebraDataset(Dataset):
    def __init__(self, root_zebra, root_horse, transform=None):
        self.root_zebra = root_zebra
        self.root_horse = root_horse
        self.transform = transform

        self.zebra_images = os.listdir(root_zebra)
        self.horse_images = os.listdir(root_horse)
        self.length_dataset = max(len(self.zebra_images), len(self.horse_images)) # 1000, 1500
        self.zebra_len = len(self.zebra_images)
        self.horse_len = len(self.horse_images)

    def __len__(self):
        return self.length_dataset

    def __getitem__(self, index):
        zebra_img = self.zebra_images[index % self.zebra_len]
        horse_img = self.horse_images[index % self.horse_len]

        zebra_path = os.path.join(self.root_zebra, zebra_img)
        horse_path = os.path.join(self.root_horse, horse_img)

        zebra_img = np.array(Image.open(zebra_path).convert("RGB"))
        horse_img = np.array(Image.open(horse_path).convert("RGB"))

        if self.transform:
            augmentations = self.transform(image=zebra_img, image0=horse_img)
            zebra_img = augmentations["image"]
            horse_img = augmentations["image0"]

        return zebra_img, horse_img

In [ ]:
class Discriminator(nn.Module):
    def __init__(self, img_size,channels =3, frontier=4, paso_de_canal=2):
        super().__init__()
        self.channels = channels
        self.img_size = img_size
        self.frontier = frontier
        self.paso_de_canal = paso_de_canal
        nc = channels
        ndf = 64
        self.layers = nn.Sequential(
            # input is (nc) x 128 x 128
            nn.Conv2d(nc,ndf,4,2,1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf) x 64 x 64
            nn.Conv2d(ndf,ndf*2,4,2,1, bias=False),
            nn.InstanceNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*2) x 32 x 32
            nn.Conv2d(ndf*2, ndf * 4, 4, 2, 1, bias=False),
            nn.InstanceNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),

            #ResidualBlock(ndf*4),
            #ResidualBlock(ndf*4),

            # state size. (ndf*4) x 16 x 16
            nn.Conv2d(ndf*4,ndf*8,4,1,1),
            nn.InstanceNorm2d(ndf*8),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*8) x 15 x 15
            nn.Conv2d(ndf*8,1,4,1,1)
            # state size. 1 x 14 x 14
        )

        return

    def forward(self, x):
      #print(f"input shape: {x.shape}")
      return self.layers(x)

def test(i):
    img_size = i
    model = Discriminator(img_size, frontier=2,channels=3).to(DEVICE)

    print(f"img_size: {img_size}")
    x = torch.randn((1,3,img_size,img_size)).to(DEVICE)

    print(model(x))
    print(torch.mean(model(x)))
    print(model(x).shape)
    print(model)
    return

test(256)


img_size: 256
tensor([[[[-3.7182e-01,  3.3837e-01,  4.9128e-01,  2.5815e-01, -3.2767e-01,
            2.9515e-01, -1.3654e-01, -1.5600e-01, -2.6314e-02, -2.6881e-01,
           -4.1931e-01,  3.9636e-01, -8.5987e-02, -4.9797e-02,  3.7645e-01,
            4.8303e-01,  2.2051e-01,  1.5544e-01, -3.9137e-01,  3.6529e-01,
           -4.9795e-02, -4.5846e-01, -2.2561e-01, -2.6378e-01, -2.6027e-01,
           -9.2613e-02, -5.0412e-01, -1.6145e-01, -6.4615e-02, -6.9973e-02],
          [-9.1032e-02,  4.2562e-02,  3.1877e-01,  1.8649e-01,  1.0428e+00,
            3.1256e-01,  3.8047e-01, -2.3131e-01,  6.5830e-01, -1.6348e-01,
            1.9473e-01,  3.9130e-01,  1.0926e-01, -1.9741e-01,  7.2733e-01,
           -6.0729e-02,  2.9468e-01, -3.5929e-01, -5.2294e-02,  1.2359e-01,
            8.8372e-02,  4.4359e-01,  8.4469e-01, -1.0356e-01,  6.7799e-01,
           -3.9814e-01, -4.4679e-01,  7.4092e-01, -3.4026e-01,  2.3135e-01],
          [ 3.3330e-01,  8.9109e-01,  4.1107e-01, -5.2675e-02,  8.9698e-

In [ ]:
import torch
import torch.nn as nn


class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, down=True, use_act=True, **kwargs):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, padding_mode="reflect", **kwargs)
            if down
            else nn.ConvTranspose2d(in_channels, out_channels, **kwargs),
            nn.InstanceNorm2d(out_channels),
            nn.ReLU(inplace=True) if use_act else nn.Identity(),
        )

    def forward(self, x):
        return self.conv(x)


class ResidualBlock(nn.Module):
    def __init__(self, channels):
        super().__init__()
        self.block = nn.Sequential(
            ConvBlock(channels, channels, kernel_size=3, padding=1),
            ConvBlock(channels, channels, use_act=False, kernel_size=3, padding=1),
        )

    def forward(self, x):
        return x + self.block(x)


class Generator(nn.Module):
    def __init__(self, img_channels, num_features=64, num_residuals=9):
        super().__init__()

        #initial
        self.model = [
            nn.Conv2d(
                img_channels,
                num_features,
                kernel_size=3,
                stride=1,
                padding=1,
                padding_mode="reflect",
            ),
            nn.InstanceNorm2d(num_features),
            nn.ReLU(inplace=True),
        ]

        #downsampling
        self.model += [
                nn.Conv2d(
                    num_features, num_features * 2, kernel_size=3,stride = 2, padding = 1),
                nn.InstanceNorm2d(num_features * 2),
                nn.ReLU(inplace=True),
                nn.Conv2d(num_features * 2, num_features * 4, kernel_size=3, stride = 2, padding = 1),
                nn.InstanceNorm2d(num_features * 4),
                nn.ReLU(inplace=True),
            ]

        #latente space
        self.model += [
            *[ResidualBlock(num_features * 4) for _ in range(num_residuals)]
        ]

        #upsampling
        self.model += [
                nn.Upsample(scale_factor=2), # --> width*2, heigh*2
                nn.Conv2d(num_features * 4, num_features * 2, 3, stride=1, padding=1),
                nn.ReLU(inplace=True),
                nn.Upsample(scale_factor=2), # --> width*2, heigh*2
                nn.Conv2d(num_features * 2, num_features, 3, stride=1, padding=1),
                nn.ReLU(inplace=True),

            ]

        self.model += [
            #nn.ReflectionPad2d(2), # Pads the input tensor using the reflection of the input boundary
            nn.Conv2d(
            num_features * 1,
            img_channels,
            kernel_size=3,
            stride=1,
            padding=1,
            padding_mode="reflect",
            ),
            #nn.ReflectionPad2d(1),
            nn.Tanh()
        ]
        self.model_sequential = nn.Sequential(*self.model)


    def forward(self, x):
        return self.model_sequential(x)


def test():
    img_channels = 3
    img_size = 256
    x = torch.randn((2, img_channels, img_size, img_size))
    gen = Generator(img_channels)
    print(gen(x).shape)
    count_parameters(gen)

if __name__ == "__main__":
    test()


torch.Size([2, 3, 256, 256])
+-------------------------------------------+------------+
|                  Modules                  | Parameters |
+-------------------------------------------+------------+
|         model_sequential.0.weight         |    1728    |
|          model_sequential.0.bias          |     64     |
|         model_sequential.3.weight         |   73728    |
|          model_sequential.3.bias          |    128     |
|         model_sequential.6.weight         |   294912   |
|          model_sequential.6.bias          |    256     |
|  model_sequential.9.block.0.conv.0.weight |   589824   |
|   model_sequential.9.block.0.conv.0.bias  |    256     |
|  model_sequential.9.block.1.conv.0.weight |   589824   |
|   model_sequential.9.block.1.conv.0.bias  |    256     |
| model_sequential.10.block.0.conv.0.weight |   589824   |
|  model_sequential.10.block.0.conv.0.bias  |    256     |
| model_sequential.10.block.1.conv.0.weight |   589824   |
|  model_sequential.10.bloc

In [ ]:
import random, torch, os, numpy as np
import torch.nn as nn
import copy

def save_checkpoint(model, optimizer, filename="my_checkpoint.pth.tar"):
    print("=> Saving checkpoint")
    checkpoint = {
        "state_dict": model.state_dict(),
        "optimizer": optimizer.state_dict(),
    }
    torch.save(checkpoint, filename)


def load_checkpoint(checkpoint_file, model, optimizer, lr):
    print("=> Loading checkpoint")
    checkpoint = torch.load(checkpoint_file, map_location=DEVICE)
    model.load_state_dict(checkpoint["state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer"])

    # If we don't do this then it will just have learning rate of old checkpoint
    # and it will lead to many hours of debugging \:
    for param_group in optimizer.param_groups:
        param_group["lr"] = lr


def seed_everything(seed=42):
    os.environ["PYTHONHASHSEED"] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [ ]:
import torch
import sys
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from torchvision.utils import save_image
import time as t


def train_fn(
    disc_H, disc_Z, gen_Z, gen_H, loader, opt_disc, opt_gen, l1, mse, d_scaler, g_scaler
):
    H_reals = 0
    H_fakes = 0
    loop = tqdm(loader, leave=True)

    for idx, (zebra, horse) in enumerate(loop):
        zebra = zebra.to(DEVICE)
        horse = horse.to(DEVICE)

        # Train Discriminators H and Z
        with torch.cuda.amp.autocast():

            fake_horse = gen_H(zebra)

            D_H_real = disc_H(horse)
            D_H_fake = disc_H(fake_horse.detach())

            H_reals += D_H_real.mean().item()
            H_fakes += D_H_fake.mean().item()

            D_H_real_loss = mse(D_H_real, torch.ones_like(D_H_real))
            D_H_fake_loss = mse(D_H_fake, torch.zeros_like(D_H_fake))
            D_H_loss = D_H_real_loss + D_H_fake_loss

            fake_zebra = gen_Z(horse)
            D_Z_real = disc_Z(zebra)
            D_Z_fake = disc_Z(fake_zebra.detach())
            D_Z_real_loss = mse(D_Z_real, torch.ones_like(D_Z_real))
            D_Z_fake_loss = mse(D_Z_fake, torch.zeros_like(D_Z_fake))
            D_Z_loss = D_Z_real_loss + D_Z_fake_loss

            # put it togethor
            D_loss = (D_H_loss + D_Z_loss) / 2

        opt_disc.zero_grad()
        d_scaler.scale(D_loss).backward()
        d_scaler.step(opt_disc)
        d_scaler.update()

        # Train Generators H and Z
        with torch.cuda.amp.autocast():
            # adversarial loss for both generators
            D_H_fake = disc_H(fake_horse)
            D_Z_fake = disc_Z(fake_zebra)
            loss_G_H = mse(D_H_fake, torch.ones_like(D_H_fake))
            loss_G_Z = mse(D_Z_fake, torch.ones_like(D_Z_fake))

            # cycle loss
            cycle_zebra = gen_Z(fake_horse)
            cycle_horse = gen_H(fake_zebra)
            cycle_zebra_loss = l1(zebra, cycle_zebra)
            cycle_horse_loss = l1(horse, cycle_horse)

            # identity loss (remove these for efficiency if you set lambda_identity=0)
            identity_zebra = gen_Z(zebra)
            identity_horse = gen_H(horse)
            identity_zebra_loss = l1(zebra, identity_zebra)
            identity_horse_loss = l1(horse, identity_horse)

            # add all togethor
            G_loss = (
                loss_G_Z
                + loss_G_H
                + cycle_zebra_loss * LAMBDA_CYCLE
                + cycle_horse_loss * LAMBDA_CYCLE
                + identity_horse_loss * LAMBDA_IDENTITY
                + identity_zebra_loss * LAMBDA_IDENTITY
            )

        opt_gen.zero_grad()
        g_scaler.scale(G_loss).backward()
        g_scaler.step(opt_gen)
        g_scaler.update()

        if idx == 0:
            timestamp = t.time()
            save_image(fake_horse * 0.5 + 0.5, f"/content/drive/MyDrive/TFG/CycleGAN_mem/output_1_{timestamp}.png")
            save_image(fake_zebra * 0.5 + 0.5, f"/content/drive/MyDrive/TFG/CycleGAN_mem/output_2_{timestamp}.png")

        loop.set_postfix(H_real=H_reals / (idx + 1), H_fake=H_fakes / (idx + 1))


def main():
    disc_H = Discriminator(256).to(DEVICE)
    disc_Z = Discriminator(256).to(DEVICE)
    gen_Z = Generator(img_channels=3, num_residuals=9).to(DEVICE)
    gen_H = Generator(img_channels=3, num_residuals=9).to(DEVICE)
    opt_disc = optim.Adam(
        list(disc_H.parameters()) + list(disc_Z.parameters()),
        lr=LEARNING_RATE,
        betas=(0.5, 0.999),
    )

    opt_gen = optim.Adam(
        list(gen_Z.parameters()) + list(gen_H.parameters()),
        lr=LEARNING_RATE,
        betas=(0.5, 0.999),
    )

    L1 = nn.L1Loss()
    mse = nn.MSELoss()

    if LOAD_MODEL:
        load_checkpoint(
            CHECKPOINT_GEN_H,
            gen_H,
            opt_gen,
            LEARNING_RATE,
        )
        load_checkpoint(
            CHECKPOINT_GEN_Z,
            gen_Z,
            opt_gen,
            LEARNING_RATE,
        )
        load_checkpoint(
            CHECKPOINT_CRITIC_H,
            disc_H,
            opt_disc,
            LEARNING_RATE,
        )
        load_checkpoint(
            CHECKPOINT_CRITIC_Z,
            disc_Z,
            opt_disc,
            LEARNING_RATE,
        )

    dataset = HorseZebraDataset(
        root_horse="Kids",
        root_zebra="Adults",
        transform=transforms,
    )
    val_dataset = HorseZebraDataset(
        root_horse="Kids",
        root_zebra="Adults",
        transform=transforms,
    )
    val_loader = DataLoader(
        val_dataset,
        batch_size=1,
        shuffle=False,
        pin_memory=True,
    )
    loader = DataLoader(
        dataset,
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=NUM_WORKERS,
        pin_memory=True,
    )
    g_scaler = torch.cuda.amp.GradScaler()
    d_scaler = torch.cuda.amp.GradScaler()

    for epoch in range(NUM_EPOCHS):
        train_fn(
            disc_H,
            disc_Z,
            gen_Z,
            gen_H,
            loader,
            opt_disc,
            opt_gen,
            L1,
            mse,
            d_scaler,
            g_scaler,
        )

        if SAVE_MODEL:
            save_checkpoint(gen_H, opt_gen, filename=CHECKPOINT_GEN_H)
            save_checkpoint(gen_Z, opt_gen, filename=CHECKPOINT_GEN_Z)
            save_checkpoint(disc_H, opt_disc, filename=CHECKPOINT_CRITIC_H)
            save_checkpoint(disc_Z, opt_disc, filename=CHECKPOINT_CRITIC_Z)


if __name__ == "__main__":
    main()

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
100%|██████████| 658/658 [02:51<00:00,  3.85it/s, H_fake=0.387, H_real=0.609]


=> Saving checkpoint
=> Saving checkpoint
=> Saving checkpoint
=> Saving checkpoint


100%|██████████| 658/658 [02:50<00:00,  3.85it/s, H_fake=0.386, H_real=0.613]


=> Saving checkpoint
=> Saving checkpoint
=> Saving checkpoint
=> Saving checkpoint


100%|██████████| 658/658 [02:50<00:00,  3.86it/s, H_fake=0.371, H_real=0.628]


...

100%|██████████| 658/658 [02:49<00:00,  3.88it/s, H_fake=0.493, H_real=0.504]


=> Saving checkpoint
=> Saving checkpoint
=> Saving checkpoint
=> Saving checkpoint


100%|██████████| 658/658 [02:49<00:00,  3.87it/s, H_fake=0.49, H_real=0.509]


=> Saving checkpoint
=> Saving checkpoint
=> Saving checkpoint
=> Saving checkpoint


 75%|███████▌  | 496/658 [02:08<00:40,  3.96it/s, H_fake=0.482, H_real=0.517]